In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

from source_code.SqlCo import Sqldd
from source_code.backpip import data_up_pip

Autosave disabled
Local Sec Env loaded


In [2]:
## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
326,AFG,Asia,Afghanistan,2020-12-14,48718.0,191.0,171.714,1969.0,4.0,9.857,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
653,ALB,Europe,Albania,2020-12-14,49191.0,661.0,786.857,1016.0,13.0,13.429,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
980,DZA,Africa,Algeria,2020-12-14,92597.0,495.0,538.857,2609.0,13.0,11.714,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [3]:
## dump the file for using it in other nb
# c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [7]:
## insert yesterday reporting to bdd
try:
    verif_data = verif_data.replace({np.nan: 'Null'})
    data_job = data_up_pip(verif_data) 
    data_job.up_baseline()
    data_job.clean_leave()
except:
    print("BDD job Failed -- Normal if you not have acces to main code")

 Insert Baseline Done !! 


In [4]:
## Load data from days -1 to evaluate the performances of model
yesterday_date = date.today() - timedelta(days=2)
yesterday_date = str(yesterday_date)

tip = Sqldd()
cnx, cursor = tip.get_bdd_co()
data_load = pd.read_sql(f"SELECT * FROM cov_aipred WHERE date='{yesterday_date}';", con=cnx) 

In [5]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    res_tempo= res_tempo.reset_index()
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
# follow_df4 = Eval(c,data_load,"Belgium")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [7]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-12-14,France,2442100.0,2433859.0,58098.0,58391.0,8241.0,-293.0
0,2020-12-14,China,94387.0,94427.0,4754.0,4754.0,-40.0,0.0
0,2020-12-14,Italy,1866280.0,1855737.0,65099.0,65011.0,10543.0,88.0
0,2020-12-14,United States,16428200.0,16519462.0,300697.0,300479.0,-91262.0,218.0
0,2020-12-14,World,73321200.0,72849650.0,1625380.0,1621154.0,471550.0,4226.0
0,2020-12-14,United Kingdom,1881300.0,1874867.0,64707.0,64500.0,6433.0,207.0
0,2020-12-14,Germany,1376720.0,1357261.0,22413.0,22634.0,19459.0,-221.0
0,2020-12-14,Iran,1125860.0,1115770.0,52939.0,52447.0,10090.0,492.0
0,2020-12-14,Turkey,2014100.0,1866345.0,16606.0,16646.0,147755.0,-40.0
0,2020-12-14,Brazil,6956600.0,6927145.0,182624.0,181835.0,29455.0,789.0


In [8]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
# rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")

In [12]:
## insert yesterday reporting to bdd
# try:
#     data_job = data_up_pip(rapport) 
#     data_job.up_rapp()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")

 Insert Rapport Done !! 
